In [1]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd
import torch
from sklearn.preprocessing import LabelEncoder
import os

cache_dir = '/media/ubuntu/5d2d9f9d-a02d-45ab-865f-3d789a0c70f0/download/'
os.environ['TRANSFORMERS_CACHE'] = cache_dir

In [ ]:
#zum Beispiel:
model_path="../01_Daten/logs/final/bert_multiclass_FT-15k/final_model/"
path_val="../01_Daten/pkl/df_val_5k-3.pkl"

In [ ]:
# Dataset Klasse definieren
class PublicationsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

def prepare_val(df):
    # Kombiniere Titel und Abstract
    df['text'] = df['title'].astype(str) + " - " + df['abstract'].astype(str)

    # Bereinigen Sie den Text
    df["text"] = df["text"].apply(clean_text).str.lower()
    # encode the labels
    df['label_encoded'] = le.fit_transform(df['class']).astype(int)
    
    df = df.sample(frac=1)
    X_val = df["text"]
    Y_val = df["label_encoded"]  



    print("Tokenizing validation data...")
    val_encodings = loaded_tokenizer(
        list(X_val), truncation=True, padding=True, max_length=512)
    
    tokenized_output = loaded_tokenizer(
    df['text'].tolist(),
    padding=True,
    truncation=True,
    max_length=512  # Typische maximale Länge für BERT-Modelle
    )
    df["text"] =  tokenized_output
    
    print("End...")
    print("creating dataset...")
    ## Dataset erstellen
    val_dataset = PublicationsDataset(val_encodings, Y_val.reset_index(drop=True))
    print("End...")
    return val_dataset,df

def clean_text(text):
    import re
    from bs4 import BeautifulSoup
    # HTML-Tags entfernen
    text = BeautifulSoup(text, "html.parser").get_text()
    text = re.sub(r"[\",\']","", text)  #  Anführungszeichen entfernen

    # 1. Mehrfache Anführungszeichen durch ein normales ' ersetzen
    text = re.sub(r"'{2,}", "'", text)

    # 2. HTML-Tags entfernen [1, 2, 3]
    # Sucht nach Mustern wie <tag>Inhalt</tag> und ersetzt sie durch einen leeren String.
    text = re.sub(r'<.*?>', '', text)

    # 3. URLs entfernen [1, 2, 3]
    # Sucht nach gängigen URL-Mustern (http/https, www.) und ersetzt sie durch einen leeren String.
    text = re.sub(r'http\S+|www\.\S+', '', text)

    # 4. E-Mail-IDs entfernen [3]
    # Sucht nach E-Mail-Mustern (Zeichenfolge@Zeichenfolge.Domain) und ersetzt sie durch einen leeren String.
    text = re.sub(r'\S*@\S*\s?', '', text)

    # 5. Zusätzliche Leerzeichen normalisieren [1, 4]
    # Teilt den Text nach Leerzeichen auf und fügt ihn mit einem einzigen Leerzeichen wieder zusammen.
    text = " ".join(text.split())

    text = re.sub(r"[\[,\]]","", text)  # Mehrfache Leerzeichen zu einem reduzieren
    
    return text

In [4]:
model_path="../01_Daten/logs/final/bert_multiclass_FT-15k/final_model/"
loaded_model_path=model_path+"model"
loaded_tokenizer_path=model_path+"tokenizer"



loaded_model = AutoModelForSequenceClassification.from_pretrained(loaded_model_path,cache_dir=cache_dir)
loaded_tokenizer = AutoTokenizer.from_pretrained(loaded_tokenizer_path, cache_dir=cache_dir)

classifier = pipeline("text-classification", model=loaded_model, tokenizer=loaded_tokenizer, device=0 if torch.cuda.is_available() else -1)
le = LabelEncoder()

# Load Validation DF
val_dataset, dfBert_val = prepare_val(pd.read_pickle(path_val))

# Klassifikation auf jede Zeile anwenden
dfBert_val['predictions_encoded'] = dfBert_val['text'].apply(lambda x:  classifier(x, truncation=True, max_length=512)[0]['label'])
dfBert_val['predictions_encoded'] = dfBert_val['predictions_encoded'].apply(lambda x: int(x[6:]))
dfBert_val['predicted_labels_named'] = dfBert_val['predictions_encoded'].apply(lambda x: le.inverse_transform([x])[0])

# Wahre Labels und vorhergesagte Labels aus dem DataFrame extrahieren
y_true_val = dfBert_val['label_encoded']
y_pred_val = dfBert_val['predictions_encoded']
print("------------------------------------------")

Device set to use cuda:0


Tokenizing validation data...
End...
creating dataset...
End...



KeyboardInterrupt



In [ ]:




print("------------------------------------------")
print("\nvalidation-set metrics (calculated from dfResults_pred):")
#F1-Score
f1_val_weighted = f1_score(y_true_val, y_pred_val, average='weighted', zero_division=0)
print(f"F1-Score: {f1_val_weighted:.4f}")
print("\n------------------------------------------")


dfResults_pred_diff = dfBert_val[dfBert_val['label_encoded'] != dfBert_val['predictions_encoded']]
if dfResults_pred_diff.empty:
    print("\nKeine Unterschiede zwischen Original- und Vorhersage-Labels im Validierungsset gefunden. Perfekte Vorhersage!")
else:
    print(f"\nAnzahl der unterschiedlichen Vorhersagen im Validierungsset: {len(dfResults_pred_diff)}")
print("\n-------------------------------------------")

In [1]:
# MISTRAL
base_path = "../01_Daten/logs/final/mistral_classifier_final_v2" # <--- BITTE ANPASSEN!

In [15]:
#path checker
import os
                
#model_path = "../01_Daten/logs/FULL_bert_multiclass_FT-15k/bert_multiclass_FT-15k/final_model/model"
if os.path.exists(base_path):
    print(f"Model directory exists at: {base_path}")
else:
    print(f"Model directory not found at: {base_path}")

Model directory exists at: ../01_Daten/logs/final/distilbert-base-multilingual-cased


In [1]:
# BERT
base_path = "../01_Daten/logs/final/bert_multiclass_FT-15k" # <--- BITTE ANPASSEN!

In [1]:
# distilbert
base_path = "../01_Daten/logs/final/distilbert-base-multilingual-cased" # <--- BITTE ANPASSEN!

In [1]:
# Pubmed
base_path = "../01_Daten/logs/final/PubMedBert_multiclass_FT-15k" # <--- BITTE ANPASSEN!

In [1]:
# agriculture
base_path = "../01_Daten/logs/final/agriculture-bert-uncased" # <--- BITTE ANPASSEN!

In [2]:
import pandas as pd
import numpy as np
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    BertTokenizer, 
    Trainer, 
    TrainingArguments, 
    DataCollatorWithPadding
)
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, classification_report # <-- Hinzugefügt
from bs4 import BeautifulSoup
import re
import os

model_name_print = base_path.split("/")
model_name_print = model_name_print[4] 

# ============== KONFIGURATION & HILFSFUNKTIONEN ==============
class PublicationsDataset(torch.utils.data.Dataset):
    """Eigene Dataset-Klasse für PyTorch."""
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item


def clean_text(text: str) -> str:
    """Bereinigt einen Text von HTML-Tags und anderen Störungen."""
    if not isinstance(text, str):
        return ""
    text = BeautifulSoup(text, "html.parser").get_text()
    text = re.sub(r"[\",\']", "", text)
    text = re.sub(r"'{2,}", "'", text)
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'http\S+|www\.\S+', '', text)
    text = re.sub(r'\S*@\S*\s?', '', text)
    text = " ".join(text.split())
    text = re.sub(r"[\[,\]]", "", text)
    return text

def prepare_prediction_data(df: pd.DataFrame, tokenizer, le: LabelEncoder) -> tuple[PublicationsDataset, pd.DataFrame]:
    df = df.sample(frac=1)
    """Bereitet einen DataFrame für die Vorhersage vor."""
    print("Kombiniere und bereinige Textspalten...")
    df['text'] = df['title'].astype(str) + " - " + df['abstract'].astype(str)
    df['class'] = df['class'].str.replace(r'ErnÃ¤hrung', 'Ernährung', regex=True)
    print("Textcleaning startet...")
    if "cased" in base_path:
        print("ohne lower...")
        df["text"] = df["text"].apply(clean_text)
    else:
        df["text"] = df["text"].apply(clean_text).str.lower()
    df['label_encoded'] = le.transform(df['class']).astype(int)
    


    X_val = df["text"].tolist()
    Y_val = df["label_encoded"]

    print("Tokenisiere Daten für die Vorhersage...")
    encodings = tokenizer(X_val, truncation=True, padding=True, max_length=512)
    
    dataset = PublicationsDataset(encodings, Y_val.reset_index(drop=True))
    print("Datensatz für die Vorhersage bereit.")
    return dataset, df

# ============== HAUPTSKRIPT ZUM AUSFÜHREN ==============

if "mistral" in base_path:
    model_path = os.path.join(base_path, "final_model")
    tokenizer_path = os.path.join(base_path, "final_model")
else:
    model_path = os.path.join(base_path, "final_model/model")
    tokenizer_path = os.path.join(base_path, "final_model/tokenizer/")

path_train = '../01_Daten/pkl/df_all_15k-2.pkl'
path_val = '../01_Daten/pkl/df_all_15k.pkl'

# --- Schritt 1: Modell und Tokenizer laden ---
print(f"Lade Modell von: {model_path}")
if not os.path.exists(model_path):
    raise FileNotFoundError(f"Modell-Verzeichnis nicht gefunden: {model_path}. Bitte den `base_path` prüfen.")
    
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
print("Modell und Tokenizer erfolgreich geladen.")

# --- Schritt 2: LabelEncoder mit Trainingsdaten wiederherstellen ---
print(f"Lade Trainingsdaten von '{path_train}', um den LabelEncoder zu fitten...")
df_train = pd.read_pickle(path_train)
df_train['class'] = df_train['class'].str.replace(r'ErnÃ¤hrung', 'Ernährung', regex=True)
le = LabelEncoder()
le.fit(df_train['class'])
print("LabelEncoder wurde mit den Trainingsdaten gefittet und ist bereit.")

# --- Schritt 3: Validierungs-Daten vorbereiten ---
print(f"Lade und bereite Validierungsdaten von '{path_val}' vor...")
df_val_original = pd.read_pickle(path_val)
val_dataset, dfBert_val = prepare_prediction_data(df_val_original, tokenizer, le)

# --- Schritt 4: Vorhersage durchführen ---
prediction_args = TrainingArguments(
    output_dir="./prediction_temp",
    per_device_eval_batch_size=64,
    report_to="none",
)
predictor = Trainer(
    model=model,
    args=prediction_args,
    data_collator=data_collator,
)
print("\nStarte die Vorhersage auf dem Validierungsdatensatz...")
predictions_output = predictor.predict(val_dataset)
print("Vorhersage abgeschlossen.")

# --- Schritt 5: Ergebnisse verarbeiten ---
print("\nVerarbeite die Vorhersage-Ergebnisse...")
predicted_scores = predictions_output.predictions
predicted_labels_encoded = np.argmax(predicted_scores, axis=1)
predicted_labels_named = le.inverse_transform(predicted_labels_encoded)

dfResults_pred = pd.DataFrame({
    'id_im_aktuellen_df': dfBert_val.index.values,
    'class_original': dfBert_val['class'].values,
    'label_encoded_original': dfBert_val['label_encoded'].values,
    'prediction_encoded': predicted_labels_encoded,
    'prediction_named': predicted_labels_named,
    'text_input': dfBert_val['text'].values
})
#print("--- Ergebnis-DataFrame (die ersten 5 Zeilen): ---")
#print(dfResults_pred.head())

# --- Schritt 6: Metriken berechnen und ausgeben ---
print(f"\n--- Metriken der Vorhersage {model_name_print} ---")
y_true = dfResults_pred['label_encoded_original']
y_pred = dfResults_pred['prediction_encoded']

# Gewichteten F1-Score berechnen
f1_weighted = f1_score(y_true, y_pred, average='weighted', zero_division=0)
print(f"Gewichteter F1-Score: {f1_weighted:.4f}")

# Detaillierten Report ausgeben
print("\nDetaillierter Klassifikations-Report:")
# `le.classes_` liefert die Namen der Klassen in der richtigen Reihenfolge
report = classification_report(y_true, y_pred, target_names=le.classes_)
print(report)

# --- Schritt 7: Ergebnisse speichern ---
#output_file_path = os.path.join(base_path, "rerun_predictions_with_metrics.pkl")
#dfResults_pred.to_pickle(output_file_path)
#print(f"\nVollständige Ergebnisse wurden in '{output_file_path}' gespeichert.")




Lade Modell von: ../01_Daten/logs/final/agriculture-bert-uncased/final_model/model
Modell und Tokenizer erfolgreich geladen.
Lade Trainingsdaten von '../01_Daten/pkl/df_all_15k-2.pkl', um den LabelEncoder zu fitten...
LabelEncoder wurde mit den Trainingsdaten gefittet und ist bereit.
Lade und bereite Validierungsdaten von '../01_Daten/pkl/df_all_15k.pkl' vor...
Kombiniere und bereinige Textspalten...
Textcleaning startet...
ohne lower...
Tokenisiere Daten für die Vorhersage...
Datensatz für die Vorhersage bereit.

Starte die Vorhersage auf dem Validierungsdatensatz...


Vorhersage abgeschlossen.

Verarbeite die Vorhersage-Ergebnisse...

--- Metriken der Vorhersage agriculture-bert-uncased ---
Gewichteter F1-Score: 0.7301

Detaillierter Klassifikations-Report:
                      precision    recall  f1-score   support

           Ernährung       0.74      0.72      0.73      3000
      Landwirtschaft       0.67      0.82      0.73      3000
             Medizin       0.76      0.71      0.74      3000
                Rest       0.68      0.61      0.64      3000
Umweltwissenschaften       0.82      0.79      0.81      3000

            accuracy                           0.73     15000
           macro avg       0.73      0.73      0.73     15000
        weighted avg       0.73      0.73      0.73     15000



--- Metriken der Vorhersage bert_multiclass_FT-15k ---
<br>
Gewichteter F1-Score: 0.7189

Detaillierter Klassifikations-Report:
                      precision    recall  f1-score   support

           Ernährung       0.72      0.70      0.71      3000
      Landwirtschaft       0.66      0.78      0.72      3000
             Medizin       0.74      0.72      0.73      3000
                Rest       0.66      0.62      0.64      3000
Umweltwissenschaften       0.83      0.77      0.80      3000

            accuracy                           0.72     15000
           macro avg       0.72      0.72      0.72     15000
        weighted avg       0.72      0.72      0.72     15000

--- Metriken der Vorhersage distilbert-base-multilingual-cased ---
<br>
Gewichteter F1-Score: 0.7123

Detaillierter Klassifikations-Report:
                      precision    recall  f1-score   support

           Ernährung       0.68      0.71      0.70      3000
      Landwirtschaft       0.73      0.70      0.71      3000
             Medizin       0.76      0.67      0.71      3000
                Rest       0.65      0.64      0.64      3000
Umweltwissenschaften       0.76      0.85      0.80      3000

            accuracy                           0.71     15000
           macro avg       0.71      0.71      0.71     15000
        weighted avg       0.71      0.71      0.71     15000

--- Metriken der Vorhersage PubMedBert_multiclass_FT-15k ---
<br>
Gewichteter F1-Score: 0.7214

Detaillierter Klassifikations-Report:
                      precision    recall  f1-score   support

           Ernährung       0.66      0.79      0.72      3000
      Landwirtschaft       0.71      0.75      0.73      3000
             Medizin       0.77      0.68      0.72      3000
                Rest       0.67      0.58      0.62      3000
Umweltwissenschaften       0.80      0.82      0.81      3000

            accuracy                           0.72     15000
           macro avg       0.73      0.72      0.72     15000
        weighted avg       0.73      0.72      0.72     15000


--- Metriken der Vorhersage agriculture-bert-uncased ---
Gewichteter F1-Score: 0.7301

Detaillierter Klassifikations-Report:
                      precision    recall  f1-score   support

           Ernährung       0.74      0.72      0.73      3000
      Landwirtschaft       0.67      0.82      0.73      3000
             Medizin       0.76      0.71      0.74      3000
                Rest       0.68      0.61      0.64      3000
Umweltwissenschaften       0.82      0.79      0.81      3000

            accuracy                           0.73     15000
           macro avg       0.73      0.73      0.73     15000
        weighted avg       0.73      0.73      0.73     15000